<a href="https://colab.research.google.com/github/gauthamhere/3.0P2/blob/master/GauthamK_Phase2_Assignment2LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
from shutil import copyfile
import keras.utils
import keras.backend as K
from keras.callbacks import ModelCheckpoint
import numpy as np
import random
import string
from keras.preprocessing import sequence
from keras.layers import LSTM,Input,Dense
from keras.models import Model

Using TensorFlow backend.


In [2]:
drive.mount('gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at gdrive


In [3]:
copyfile("gdrive/My Drive/lstm_assignment/wonderland.txt",'wonderland.txt')

'wonderland.txt'

In [0]:
def load_characters(source, length=None, limit=None,):
    
    # Reading raw text from source and destination files
    f = open(source, 'r')
    x_data = f.read()
    f.close()

    print('raw data read')
    
    x_data = x_data.lower()
    
    #Removing punctutations
    puncts = string.punctuation
    x_data = str(x_data)
    for punct in puncts:
      if punct in x_data:
        x_data = x_data.replace(punct, '')

    if limit is not None:
        x_data = x_data[:limit]

    # Splitting raw text into array of sequences
    if length is None:
        x = [list(line) for line in x_data.split('\n') if len(line) > 0]
    else:
        x = [list(chunk) for chunk in chunks(x_data, length)]

    # Creating the vocabulary set with the most common characters (leaving room for PAD, START, UNK)
    chars = set()
    for line in x:
        for char in line:
            chars.add(char)

    # Creating an array of words from the vocabulary set, we will use this array as index-to-word dictionary
    i2c = list(chars)

    # Creating the word-to-index dictionary from the array created above
    c2i = {word:ix for ix, word in enumerate(i2c)}

    # Converting each word to its index value
    for i, sentence in enumerate(x):
        for j, word in enumerate(sentence):
            if word in c2i:
                x[i][j] = c2i[word]
            else:
                x[i][j] = c2i['<UNK>']

    return x, c2i, i2c

In [0]:
def sample(preds, temperature=1.0):
    """
    Sample an index from a probability vector
    :param preds:
    :param temperature:
    :return:
    """

    preds = np.asarray(preds).astype('float64')

    if temperature == 0.0:
        return np.argmax(preds)

    preds = np.log(preds) / temperature

    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    probas = np.random.multinomial(1, preds, 1)

    return np.argmax(probas)


In [0]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [7]:
x, char_to_ix, ix_to_char = load_characters('wonderland.txt',length=100)

raw data read


In [8]:
x_max_len = max([len(sentence) for sentence in x])
numchars = len(ix_to_char)
print(numchars, ' distinct characters found')

28  distinct characters found


In [0]:
x = sequence.pad_sequences(x, x_max_len, padding='post', truncating='post')

In [0]:
def decode(seq):
        return ''.join(ix_to_char[id] for id in seq)

In [0]:
n = x.shape[0]

x_in  = np.concatenate([np.ones((n, 1)), x[:, :-1]], axis=1)  
x_out = x
assert x_in.shape == x_out.shape

In [0]:
def to_categorical(batch, num_classes):
    
    b, l = batch.shape

    out = np.zeros((b, l, num_classes))

    for i in range(b):
        seq = batch[0, :]
        out[i, :, :] = keras.utils.to_categorical(seq, num_classes=num_classes)

    return out

In [0]:
#  convert from integer sequences to sequences of one-hot vectors
x_in = to_categorical(x_in, numchars)
x_out =to_categorical(x_out, numchars)  

In [14]:
input = Input(shape=(None, numchars))
h = LSTM(256,dropout =0.1, return_sequences=True)(input)
h = LSTM(256, return_sequences=True)(h)
out = Dense(numchars, activation='softmax')(h)
model = Model(input, out)

W0726 17:32:55.337434 140487774812032 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0726 17:32:55.373669 140487774812032 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 17:32:55.381261 140487774812032 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0726 17:32:55.574857 140487774812032 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0726 17:32:55.587419 

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 28)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 256)         291840    
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 256)         525312    
_________________________________________________________________
dense_1 (Dense)              (None, None, 28)          7196      
Total params: 824,348
Trainable params: 824,348
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

W0726 17:32:56.196208 140487774812032 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0726 17:32:56.224288 140487774812032 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [0]:
def generate_seq(model : Model,seed,numchars,size,temperature=1.0):
    
    ls = seed.shape[0]

    tokens = np.concatenate([seed, np.zeros(size - ls)])

    # convert the integer sequence to a categorical one
    toh = to_categorical(tokens[None, :], numchars)

    for i in range(ls, size-1):

        # predict next characters (for the whole sequence)
        probs = model.predict(toh)

        # Extract the i-th probability vector and sample an index from it
        next_token = sample(probs[0, i-1, :], temperature)

        tokens[i] = next_token

        # update the one-hot encoding
        toh[0, i, 0] = 0
        toh[0, i, next_token] = 1

    return [int(t) for t in tokens]

In [0]:
CHECK = 5
gen_length = 100
out_every = 25 #Generate output for every 5 epochs

def generate(epoch):
        if epoch % out_every == 0 and epoch > 0:
            for i in range(CHECK):
                b = random.randint(0, n - 1)

                seed = x[b, :20]
                seed = np.insert(seed, 0, 1)
                gen = generate_seq(model, seed, numchars, gen_length)

                print('Seed:', decode(seed), '\n\nGenerated text:', decode(gen[len(seed):]))
                print()

    # Train the model
generate_stuff = keras.callbacks.LambdaCallback(on_epoch_end=lambda epoch, logs: generate(epoch))

In [0]:
# define the checkpoint
filepath="gdrive/My Drive/lstm_assignment/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint,generate_stuff]

In [0]:
epochs = 100
batch_size = 512

In [21]:
#Training for 100 epochsa
model.fit(x_in, x_out, epochs=epochs, batch_size=batch_size, callbacks=callbacks_list)

W0726 17:32:56.449955 140487774812032 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1401/1401 [==============================] - 6s 4ms/step - loss: 3.3037

Epoch 00001: loss improved from inf to 3.30368, saving model to gdrive/My Drive/lstm_assignment/weights-improvement-01-3.3037-bigger.hdf5
Epoch 2/100
1401/1401 [==============================] - 1s 609us/step - loss: 3.0793

Epoch 00002: loss improved from 3.30368 to 3.07930, saving model to gdrive/My Drive/lstm_assignment/weights-improvement-02-3.0793-bigger.hdf5
Epoch 3/100
1401/1401 [==============================] - 1s 620us/step - loss: 2.8803

Epoch 00003: loss improved from 3.07930 to 2.88029, saving model to gdrive/My Drive/lstm_assignment/weights-improvement-03-2.8803-bigger.hdf5
Epoch 4/100
1401/1401 [==============================] - 1s 596us/step - loss: 2.7694

Epoch 00004: loss improved from 2.88029 to 2.76940, saving model to gdrive/My Drive/lstm_assignment/weights-improvement-04-2.7694-bigger.hdf5
Epoch 5/100
1401/1401 [==============================] - 1s 624us/step - loss: 2.7272

Epo

In [0]:
model.load_weights("gdrive/My Drive/lstm_assignment/weights-improvement-93-0.1973-bigger.hdf5")

In [25]:
gen_length=500  #Predicting 500 characters

b = random.randint(0, n - 1)
seed = x[b, :40]
seed = np.insert(seed, 0, 1)
  
gen = generate_seq(model,seed, numchars, gen_length)

print('Seed:', decode(seed), '\n\nGenerated text:', decode(gen[len(seed):]))

Seed: ned her arms round it as far as they
woul 

Generated text: ff sttting b heer sstter
o  hhe bank an oof havvng nnthing ttt t veer  tiiedoo  iitting by her sister
on h e bank aandoo  hvving nothing tttet veeryttieed  f sitting  y her iister
nn th  bann andnoo  aviiin nnhhing tttg  veveyttired  f sittinn bb her sisstr

onthe  bank add  f having nothing ttttt vrey  iire  o ssitiing yy her sister
on the aann and of having nnthig  ttt t very tiired o  sitting by her ssster
on the bank nnn of havinn nothiig tttt tvre ya
